In [ ]:
%%timeit -r 10 -n 30

import psycopg2
import pandas as pd

def connect_to_database():
    # Funzione per connettersi al database
    try:
        connection = psycopg2.connect(
            user="il_tuo_utente",
            password="la_tua_password",
            host="il_tuo_host",
            port="la_tua_porta",
            database="il_tuo_database"
        )
        return connection
    except psycopg2.Error as e:
        print("Errore durante la connessione al database:", e)
        return None

def close_connection(connection):
    # Funzione per chiudere la connessione al database
    if connection:
        connection.close()

def execute_query(connection, query):
    # Funzione per eseguire la query e restituire i risultati
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            return result
    except psycopg2.Error as e:
        print("Errore durante l'esecuzione della query:", e)
        return None

def main():
    # Connessione al database
    connection = connect_to_database()

    # Verifica della connessione
    if not connection:
        return

    try:
        # Query SQL
        sql_query = """
        SELECT *
        FROM "apparecchiatureelettroniche"
        WHERE CAST(SPLIT_PART("risoluzioneschermo", 'x', 2) AS INT) > 1080
        ORDER BY "dataproduzione" DESC;
        """

        # Esecuzione della query
        results = execute_query(connection, sql_query)

        if results:
            # Creazione di un dataframe pandas con i risultati
            columns = [desc[0] for desc in connection.cursor().description]
            df = pd.DataFrame(results, columns=columns)

            # Stampa a video e salvataggio del dataframe
            print("Risultati della query:")
            print(df)

            # Salva il dataframe in un file CSV, se necessario
            df.to_csv("risultati_query.csv", index=False)

    finally:
        # Chiusura della connessione al database alla fine dell'esecuzione
        close_connection(connection)

if __name__ == "__main__":
    main()
